In [2]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=859c0183825267a8988a0ef5efec2dbd2f0361aff4532665d5405325d35645be
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [62]:
!pip install --upgrade prince


In [6]:
!pip install prince


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.6/415.6 kB 6.1 MB/s eta 0:00:00


In [116]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
import numpy as np
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [149]:
def dim_red(mat, p, method):
  if method == 'T-SNE' :
    model = TSNE(n_components=p)
    red_mat = model.fit_transform(mat)
  else:
        raise Exception("Please select one of the three methods : APC, T-SNE, UMAP")
  return red_mat

In [151]:
def clust(mat, k):
    np.array(mat)
    kmeans = KMeans(n_clusters=k)
    cluster_labels = kmeans.fit_predict(mat)
    return cluster_labels

    return pred


In [152]:
# import data
ng20 = fetch_20newsgroups(subset='test')
corpus = ng20.data[:2000]
labels = ng20.target[:2000]
k = len(set(labels))

# embedding
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(corpus)

In [153]:
method = 'T-SNE'
red_emb = dim_red(embeddings, 3, method)  # 3 dimensions for t-SNE


In [154]:
# Perform clustering
# Perform clustering
pred = clust(red_emb, k)


# Evaluate clustering results
nmi_score = normalized_mutual_info_score(pred, labels)
ari_score = adjusted_rand_score(pred, labels)
acc = accuracy_score(labels, pred)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [155]:
print(f'Method :{method} \nNMI: {nmi_score:.2f} \nARI: {ari_score:.2f}\nAccuracy: {acc:.2f}\n')

Method :T-SNE 
NMI: 0.41 
ARI: 0.24
Accuracy: 0.03



In [139]:
# Perform t-SNE and clustering for each fold in the cross-validation
n_splits = 5  # You can adjust the number of splits
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_index, test_index) in enumerate(skf.split(embeddings, labels)):
    X_train, X_test = embeddings[train_index], embeddings[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    method = 't-SNE'
    red_emb = dim_red(X_train, 3,method)  # 3 dimensions for t-SNE

    pred = clust(red_emb, k)

    # Evaluate clustering results
    nmi_score = normalized_mutual_info_score(pred, y_train)
    ari_score = adjusted_rand_score(pred, y_train)
    acc = accuracy_score(y_train, pred)
    print(f'Method :{method} ')
    print(f'Fold: {fold+1}')
    print(f'\nNMI: {nmi_score:.2f} \nARI: {ari_score:.2f}\nAccuracy: {acc:.2f}\n')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Method :t-SNE 
Fold: 1

NMI: 0.42 
ARI: 0.23
Accuracy: 0.06



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Method :t-SNE 
Fold: 2

NMI: 0.42 
ARI: 0.24
Accuracy: 0.08



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Method :t-SNE 
Fold: 3

NMI: 0.40 
ARI: 0.21
Accuracy: 0.04



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Method :t-SNE 
Fold: 4

NMI: 0.44 
ARI: 0.26
Accuracy: 0.04

Method :t-SNE 
Fold: 5

NMI: 0.41 
ARI: 0.24
Accuracy: 0.02



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [156]:
import plotly.express as px

# Assuming red_emb is a DataFrame containing the reduced embeddings after t-SNE
# Assuming pred is the predicted cluster labels after clustering

# Create an interactive scatter plot
fig = px.scatter(red_emb, x=0, y=1, color=pred, title='Interactive Scatter Plot after t-SNE and Clustering',
                 labels={'0': 'Dimension 1', '1': 'Dimension 2'},
                 color_continuous_scale='viridis')

fig.show()


In [128]:
import plotly.express as px

# Assuming red_emb is a DataFrame containing the reduced embeddings after t-SNE
# Assuming pred is the predicted cluster labels after clustering

# Create an interactive 3D scatter plot
fig_3d = px.scatter_3d(red_emb, x=0, y=1, z=2, color=pred, title='Interactive 3D Scatter Plot after t-SNE and Clustering',
                      labels={'0': 'Dimension 1', '1': 'Dimension 2', '2': 'Dimension 3'},
                      color_continuous_scale='viridis')

fig_3d.show()
